### This code is a new branch where CSAT data that use is after analysis (Taking Out The Unnecessary Data)

In [2]:
import gspread
import pandas as pd
from datetime import date
from google.oauth2.service_account import Credentials

## Pipeline A

In [3]:
SERVICE_ACCOUNT_FILE = '../../key/credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

sheet = client.open("Chatbot Ops Report").worksheet("[CSAT After Take Out]")
data = sheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Date               591 non-null    object
 1   Total Good Survey  591 non-null    object
 2   Total Bad Survey   591 non-null    object
 3   Total Responden    591 non-null    object
 4   Total Rating       591 non-null    object
 5   CSAT [Before]      591 non-null    object
 6   CSAT [After]       591 non-null    object
 7   %                  591 non-null    object
 8                      591 non-null    object
 9   Date               591 non-null    object
 10  Total Good Survey  591 non-null    object
 11  Total Bad Survey   591 non-null    object
 12  Total Responden    591 non-null    object
 13  Total Rating       591 non-null    object
 14  CSAT               591 non-null    object
dtypes: object(15)
memory usage: 69.4+ KB


In [10]:
# Copying the DF
df_before = (
    df.copy()
    .iloc[:, [0,3,4,5]]
    .drop_duplicates(subset='Date')
)

df_after = (
    df.copy()
    .iloc[:, [9,12,13,6]]
    .drop_duplicates(subset='Date')
)

# Changing the Date data type
df_before['Date'] = pd.to_datetime(
    df_before['Date'], format='%d-%b-%Y', errors='coerce'
)

df_after['Date'] = pd.to_datetime(
    df_after['Date'], format='%d-%b-%Y', errors='coerce'
)

# dropping NA
df_before = df_before.dropna(subset=['Date'])
df_after = df_after.dropna(subset=['Date'])

df_before.info()

<class 'pandas.core.frame.DataFrame'>
Index: 589 entries, 0 to 590
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             589 non-null    datetime64[ns]
 1   Total Responden  589 non-null    object        
 2   Total Rating     589 non-null    object        
 3   CSAT [Before]    589 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 23.0+ KB


In [11]:
df_after.info()

<class 'pandas.core.frame.DataFrame'>
Index: 588 entries, 0 to 590
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             588 non-null    datetime64[ns]
 1   Total Responden  588 non-null    object        
 2   Total Rating     588 non-null    object        
 3   CSAT [After]     588 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 23.0+ KB


In [13]:
df_before.merge(df_after, on='Date', how='left', suffixes=('_Before', '_After'))

,Date,Total Responden_Before,Total Rating_Before,CSAT [Before],Total Responden_After,Total Rating_After,CSAT [After]
0,2024-03-01,59,196,3.32,50,184,3.68
1,2024-03-02,49,162,3.31,42,154,3.67
2,2024-03-03,40,152,3.80,38,150,3.95
3,2024-03-04,42,152,3.62,37,147,3.97
4,2024-03-05,54,190,3.52,51,186,3.65
...,...,...,...,...,...,...,...
584,2025-10-16,23,94,4.09,21,92,4.38
585,2025-10-17,19,71,3.74,17,69,4.06
586,2025-10-18,14,61,4.36,13,60,4.62
587,2025-10-19,28,76,2.71,22,69,3.14


In [ ]:
# export to csv
df_before.to_csv('../../dataset_kula/csat_before_takeout.csv')
df_after.to_csv('../../dataset_kula/csat_after_takeout.csv')